In [5]:
import pandas as pd
data = pd.read_csv('physionet.org/files/mimiciii/1.4/NOTEEVENTS.csv.gz', nrows=1000, compression='gzip',usecols=['TEXT'],
                   error_bad_lines=False)
# Just get 1000 data points and only the text columns

/tmp/ipykernel_11970/3266290345.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv('physionet.org/files/mimiciii/1.4/NOTEEVENTS.csv.gz', nrows=1000, compression='gzip',usecols=['TEXT'],


In [6]:
import torch
import numpy as np

In [7]:
torch.version.cuda

'11.7'

In [8]:
key_word = ['very','pleasant'] #
pattern = '|'.join(key_word)

In [9]:
data['check'] = data['TEXT'].str.contains(pattern)

In [10]:
data.check.value_counts()

True     742
False    258
Name: check, dtype: int64

In [11]:
data.TEXT.head()

0    Admission Date:  [**2151-7-16**]       Dischar...
1    Admission Date:  [**2118-6-2**]       Discharg...
2    Admission Date:  [**2119-5-4**]              D...
3    Admission Date:  [**2124-7-21**]              ...
4    Admission Date:  [**2162-3-3**]              D...
Name: TEXT, dtype: object

In [12]:
data['TEXT'] = data.TEXT.str.replace('[^a-zA-Z1-9]', ' ') # Basic cleaning to remove *or -, can be further customized later

/tmp/ipykernel_11970/2491769875.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data['TEXT'] = data.TEXT.str.replace('[^a-zA-Z1-9]', ' ') # Basic cleaning to remove *or -, can be further customized later


In [13]:
data['TEXT'].head()

0    Admission Date      2151 7 16          Dischar...
1    Admission Date      2118 6 2          Discharg...
2    Admission Date      2119 5 4                 D...
3    Admission Date      2124 7 21                 ...
4    Admission Date      2162 3 3                 D...
Name: TEXT, dtype: object

## 1: Doc_to_vec

In [14]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /home/gwan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [15]:
sentences = data.TEXT.values
tokenized_sent = []

for s in sentences:
    tokenized_sent.append(word_tokenize(s.lower()))
print(tokenized_sent[0])

['admission', 'date', '2151', '7', '16', 'discharge', 'date', '2151', '8', '4', 'service', 'addendum', 'radiologic', 'studies', 'radiologic', 'studies', 'also', 'included', 'a', 'chest', 'ct', 'which', 'confirmed', 'cavitary', 'lesions', 'in', 'the', 'left', 'lung', 'apex', 'consistent', 'with', 'infectious', 'process', 'tuberculosis', 'this', 'also', 'moderate', 'sized', 'left', 'pleural', 'effusion', 'head', 'ct', 'head', 'ct', 'showed', 'no', 'intracranial', 'hemorrhage', 'or', 'mass', 'effect', 'but', 'old', 'infarction', 'consistent', 'with', 'past', 'medical', 'history', 'abdominal', 'ct', 'abdominal', 'ct', 'showed', 'lesions', 'of', 't1', 'and', 'sacrum', 'most', 'likely', 'secondary', 'to', 'osteoporosis', 'these', 'can', 'be', 'followed', 'by', 'repeat', 'imaging', 'as', 'an', 'outpatient', 'first', 'name8', 'namepattern2', 'first', 'name4', 'namepattern1', '1775', 'last', 'name', 'namepattern1', 'm', 'd', 'md', 'number', '1', '1776', 'dictated', 'by', 'hospital', '18', '7', 

In [16]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
tagged_data = [TaggedDocument(d, [i]) for i, d in enumerate(tokenized_sent)]

In [20]:
model_d2v = Doc2Vec(tagged_data, vector_size = 200, window = 2, min_count = 10, epochs = 100)

'''
vector_size = Dimensionality of the feature vectors.
window = The maximum distance between the current and predicted word within a sentence.
min_count = Ignores all words with total frequency lower than this.
alpha = The initial learning rate.
'''

'\nvector_size = Dimensionality of the feature vectors.\nwindow = The maximum distance between the current and predicted word within a sentence.\nmin_count = Ignores all words with total frequency lower than this.\nalpha = The initial learning rate.\n'

In [48]:
data['tokens'] = tokenized_sent

In [49]:
data['emb'] = data['tokens'].apply(model_d2v.infer_vector)

In [50]:
data.head()

,TEXT,check,d2v_emb,tokens,emb
0,Admission Date 2151 7 16 Dischar...,False,1,"[admission, date, 2151, 7, 16, discharge, date...","[-0.1348736, -0.08597029, 0.8260691, -0.941025..."
1,Admission Date 2118 6 2 Discharg...,False,0,"[admission, date, 2118, 6, 2, discharge, date,...","[-0.36500144, -1.0892587, 2.0856843, 1.5134856..."
2,Admission Date 2119 5 4 D...,True,0,"[admission, date, 2119, 5, 4, discharge, date,...","[0.37884852, -2.0442128, 2.214921, 1.3320479, ..."
3,Admission Date 2124 7 21 ...,True,0,"[admission, date, 2124, 7, 21, discharge, date...","[1.6855615, -0.68606406, 0.42220047, -0.020631..."
4,Admission Date 2162 3 3 D...,True,0,"[admission, date, 2162, 3, 3, discharge, date,...","[-0.46287867, 0.25666678, 0.17845446, -0.16059..."


## Transfomer 

In [3]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('TimKond/S-BioLinkBert-MedQuAD')

In [17]:
embeddings = model.encode(sentences)
print(embeddings)

[[ 0.277509    0.26719996 -0.0105282  ... -0.12347386  1.0648662
  -0.17493397]
 [ 0.03479018  0.18932167 -0.02936742 ... -0.05296332  0.73083
  -0.21440667]
 [-0.02235499  0.33601063  0.13468382 ... -0.03416088  0.07084354
  -0.35041183]
 ...
 [-0.1237298   0.33057767  0.28832722 ... -0.15255748  0.23030174
  -0.4573601 ]
 [ 0.04183475  0.24472262  0.10942968 ... -0.05232212  0.44331408
  -0.23331973]
 [-0.05767366  0.21234523  0.21282771 ...  0.09578045 -0.10990594
  -0.38771906]]


In [56]:
from nltk.cluster import KMeansClusterer
import nltk

def clustering_question(data,emb_col,NUM_CLUSTERS = 2):

    X = np.array(data[emb_col].tolist())

    kclusterer = KMeansClusterer(
        NUM_CLUSTERS, distance=nltk.cluster.util.cosine_distance,
        repeats=25,avoid_empty_clusters=True)

    assigned_clusters = kclusterer.cluster(X, assign_clusters=True)

    data['cluster'] = pd.Series(assigned_clusters, index=data.index)
    data['centroid'] = data['cluster'].apply(lambda x: kclusterer.means()[x])

    return data

In [57]:
result_d2v = clustering_question(data)

In [60]:
result_d2v.groupby(["cluster", "check"]).size()

cluster  check
0        False     88
         True     611
1        False    170
         True     131
dtype: int64